In [2]:
# importing Qiskit
import numpy as np
from numpy import pi
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit_ibm_provider import IBMProvider

In [ ]:
api_token='IBM TOKEN'

IBMProvider.save_account(api_token, overwrite=True)
provider = IBMProvider()
print("Available backends:", provider.backends())

In [ ]:
def generateRandomState(n,delta):
     nQubits = int((4 + delta)*n) 
     basis = np.random.randint(2, size=(nQubits,))
     return basis

In [3]:
class Alice:
    def prepareState(n,delta):
        alice_data_state=generateRandomState(n,delta)
        alice_basis_state=generateRandomState(n,delta)
        return (alice_data_state,alice_basis_state)    
    

In [ ]:
class Bob:
    def getBasis(n,delta):
        return generateRandomState(n,delta)

class Eve:
    def getBasis(n,delta):
        return generateRandomState(n,delta)


In [ ]:
def BB84Algorithm(key_length):
    n = key_length
    # error slack
    delta = 0
    # number of physical qubits
    nQubits = int((4 + delta)*n)

    bb_circuit= QuantumCircuit(nQubits,nQubits)
    bb_circuit_eve= QuantumCircuit(nQubits,nQubits)

    alice = Alice()
    bob = Bob()
    eve = Eve()
    (alice_data_state,alice_basis_state) = alice.prepareState(n,delta)

    for idx in range(nQubits):
        if(alice_data_state[idx]==1):
            bb_circuit.x(idx)
            bb_circuit_eve.x(idx)

    for idx in range(nQubits):
        if(alice_basis_state[idx]==1):
            bb_circuit.h(idx)
            bb_circuit_eve.h(idx)

    bob_basis_state = bob.getBasis(n,delta)

    eve_basis_state = eve.getBasis(n,delta)

    for idx in range(nQubits):
        if(eve_basis_state[idx]==1):
            bb_circuit_eve.h(idx)

    for idx in range(nQubits):
        if(bob_basis_state[idx]==1):
            bb_circuit.h(idx)
            bb_circuit_eve.h(idx)

    for idx in range(nQubits):
        bb_circuit.measure(idx,idx)
        bb_circuit_eve.measure(idx,idx)
    
    return (bb_circuit,bb_circuit_eve,alice_data_state,alice_basis_state,bob_basis_state)